In [1]:
!pip install -U natasha
!pip install ipymarkup

In [2]:
!pip install stop_words

In [3]:
sents = ['Посол Израиля на Украине Йоэль Лион признался, что пришел в шок, узнав о решении властей Львовской области объявить 2019 год годом лидера запрещенной в России Организации украинских националистов (ОУН) Степана Бандеры.',
 'Свое заявление он разместил в Twitter.',
 '«Я не могу понять, как прославление тех, кто непосредственно принимал участие в ужасных антисемитских преступлениях, помогает бороться с антисемитизмом и ксенофобией.',
 'Украина не должна забывать о преступлениях, совершенных против украинских евреев, и никоим образом не отмечать их через почитание их исполнителей», — написал дипломат.',
 '11 декабря Львовский областной совет принял решение провозгласить 2019 год в регионе годом Степана Бандеры в связи с празднованием 110-летия со дня рождения лидера ОУН (Бандера родился 1 января 1909 года).',
 'В июле аналогичное решение принял Житомирский областной совет.',
 'В начале месяца с предложением к президенту страны Петру Порошенко вернуть Бандере звание Героя Украины обратились депутаты Верховной Рады.',
 'Парламентарии уверены, что признание Бандеры национальным героем поможет в борьбе с подрывной деятельностью против Украины в информационном поле, а также остановит «распространение мифов, созданных российской пропагандой».',
 'Степан Бандера (1909-1959) был одним из лидеров Организации украинских националистов, выступающей за создание независимого государства на территориях с украиноязычным населением.',
 'В 2010 году в период президентства Виктора Ющенко Бандера был посмертно признан Героем Украины, однако впоследствии это решение было отменено судом.']

In [4]:
from natasha import (
    Segmenter,
    MorphVocab,
    NamesExtractor,
    AddrExtractor
)

from stop_words import get_stop_words
import re

ru_stopwords = get_stop_words('ru')

In [5]:
mv = MorphVocab()

In [6]:
ne = NamesExtractor(mv)
ae = AddrExtractor(mv)

In [7]:
text = sents[0]
ents = list(ne(text)) + list(ae(text))
ents[0]

Match(
    start=0,
    stop=13,
    fact=Name(
        first='Израиля',
        last='Посол',
        middle=None
    )
)

In [13]:
prepare_ents = []
for ent in ents:
    prepare_ents.append((
      ent.start,
      ent.stop,
      ent.fact.__class__.__name__
  ))

In [9]:
from ipymarkup import show_span_line_markup
show_span_line_markup(
    text,
    prepare_ents
)

# 1. Отсмотрев результаты извлечения, написать правила фильтрации ложно-положительных сущностей - 1.5 балла

In [26]:
def markup_text_from_ents_new(text):
    ents = list(ne(text)) + list(ae(text))
    
    prepare_ents = []

    for ent in ents:

        if hasattr(ent.fact, 'last'):
            token = ent.fact.last if ent.fact.last else ent.fact.first
        else:
            token = ent.fact.value
        i = re.findall(r'[А-ЯA-Z]{1}', str(token)) # находим слова, начинающиеся с заглавной буквы
    
        if token.lower() not in ru_stopwords and i: # удаляем стоп слова
            prepare_ents.append((
              ent.start,
              ent.stop,
              ent.fact.__class__.__name__,
          ))
    
    show_span_line_markup(
                                text,
                                prepare_ents
                                        )

In [27]:
markup_text_from_ents_new(text)

# 2. Визуализировать на N примерах до и после (при этом сравнивать результаты должно быть просто и на глаз) - 1 балл

In [14]:
for sent in sents:
    print('ВАРИАНТ_1: ')
    show_span_line_markup(text, prepare_ents)
    print('ВАРИАНТ_2: ')
    markup_text_from_ents_new(sent)
    print("\n\n")
    print()

ВАРИАНТ_1: 


ВАРИАНТ_2: 






ВАРИАНТ_1: 


ВАРИАНТ_2: 






ВАРИАНТ_1: 


...

ВАРИАНТ_2: 






ВАРИАНТ_1: 


ВАРИАНТ_2: 






ВАРИАНТ_1: 


ВАРИАНТ_2: 


# 3. Написать функцию сортировки списка сущностей:

*   использовать полный перебор в дальнейшем  - 0 баллов
*   python sort + lambda - 1 балл
*   custom sort - 3 балла

In [15]:
ents = list(ne(sents[5])) + list(ae(sents[5]))

sorted(ents, key=lambda x: x.fact.last, reverse=True)

[Match(
     start=56,
     stop=61,
     fact=Name(
         first=None,
         last='совет',
         middle=None
     )
 ),
 Match(
     start=2,
     stop=6,
     fact=Name(
         first=None,
         last='июле',
         middle=None
     )
 ),
 Match(
     start=34,
     stop=45,
     fact=Name(
         first=None,
         last='Житомирский',
         middle=None
     )
 ),
 Match(
     start=0,
     stop=1,
     fact=Name(
         first=None,
         last='В',
         middle=None
     )
 )]

# 4. Написать правила (минимум 2) по извлечению структурированной информации, (например: имя - локация) - 2 балла

In [24]:
def return_new_ents(sent):
    new_ents = re.findall(r'\b[А-ЯA-Z]{1}\w+', sent) 
    new_ents = [ent for ent in new_ents if ent.lower() not in ru_stopwords]
    return new_ents

In [25]:
for sent in sents:
    print('Text:', sent)
    print('NAMES_LOC: ', ' '.join(return_new_ents(sent)))
    print("\n\n")
    print()

Text: Посол Израиля на Украине Йоэль Лион признался, что пришел в шок, узнав о решении властей Львовской области объявить 2019 год годом лидера запрещенной в России Организации украинских националистов (ОУН) Степана Бандеры.
NAMES_LOC:  Посол Израиля Украине Йоэль Лион Львовской России Организации ОУН Степана Бандеры




Text: Свое заявление он разместил в Twitter.
NAMES_LOC:  Twitter




Text: «Я не могу понять, как прославление тех, кто непосредственно принимал участие в ужасных антисемитских преступлениях, помогает бороться с антисемитизмом и ксенофобией.
NAMES_LOC:  




Text: Украина не должна забывать о преступлениях, совершенных против украинских евреев, и никоим образом не отмечать их через почитание их исполнителей», — написал дипломат.
NAMES_LOC:  Украина




Text: 11 декабря Львовский областной совет принял решение провозгласить 2019 год в регионе годом Степана Бандеры в связи с празднованием 110-летия со дня рождения лидера ОУН (Бандера родился 1 января 1909 года).
NAMES_LO

# 5. Добавить еще экстраткоры из наташи (минимум +2 экстратора) и дописать правила фильтрации (если нужны) и правила структурирования - 1.5 балла

In [18]:
from natasha import DatesExtractor, MoneyExtractor

In [19]:
de = DatesExtractor(mv)
me = MoneyExtractor(mv)

In [20]:
ents_add = list(ne(text)) + list(ae(text)) + list(de(text)) + list(me(text))
ents[1]

Match(
    start=2,
    stop=6,
    fact=Name(
        first=None,
        last='июле',
        middle=None
    )
)

In [28]:
def markup_text_from_ents_new(text):
    ents = list(ne(text)) + list(ae(text))
    ents_new = list(de(text)) + list(me(text))
    prepare_ents = []

    for ent in ents:
        
        if hasattr(ent.fact, 'last'):
            token = ent.fact.last if ent.fact.last else ent.fact.first
        else:
            token = ent.fact.value

        i = re.findall(r'[А-ЯA-Z]{1}', str(token)) 

        if token.lower() not in ru_stopwords and i:
            prepare_ents.append((
              ent.start,
              ent.stop,
              ent.fact.__class__.__name__,
          ))
            
    for ent in ents_new:
                prepare_ents.append((
                            ent.start,
                            ent.stop,
                            ent.fact.__class__.__name__
                          ))

    
    show_span_line_markup(
                                text,
                                prepare_ents
                                        )

In [29]:
markup_text_from_ents_new(text)

In [30]:
ents_add = list(ne(text)) + list(ae(text)) + list(de(text)) + list(me(text))
sorted(ents, key=lambda x: x.fact.last, reverse=True)

[Match(
     start=56,
     stop=61,
     fact=Name(
         first=None,
         last='совет',
         middle=None
     )
 ),
 Match(
     start=2,
     stop=6,
     fact=Name(
         first=None,
         last='июле',
         middle=None
     )
 ),
 Match(
     start=34,
     stop=45,
     fact=Name(
         first=None,
         last='Житомирский',
         middle=None
     )
 ),
 Match(
     start=0,
     stop=1,
     fact=Name(
         first=None,
         last='В',
         middle=None
     )
 )]

## Задание
1. Отсмотрев результаты извлечения, написать правила фильтрации ложно-положительных сущностей - 1.5 балла

2. Визуализировать на N примерах до и после (при этом сравнивать результаты должно быть просто и на глаз) - 1 балл

3. Написать функцию сортировки списка сущностей:

*   использовать полный перебор в дальнейшем  - 0 баллов
*   python sort + lambda - 1 балл
*   custom sort - 3 балла

4. Написать правила (минимум 2) по извлечению структурированной информации, (например: имя - локация) - 2 балла

5. Добавить еще экстраткоры из наташи (минимум +2 экстратора) и дописать правила фильтрации (если нужны) и правила структурирования - 1.5 балла

6. Оценить по сложности, написанные алгоритмы и весь пайплайн в целом - 1 балл

7. Ускорить и показать ускорение - 2 балла


